In [1]:
df = pd.read_csv('population_communes.csv', dtype=str)

# Cleaning:
df['value'] = df.value.astype(int)
df = df.rename(columns={
    'Code région': 'code_region',
    'Code département': 'code_dept',
})

# Aggrégation à la maille département:
df = df.groupby(['code_region', 'code_dept', 'variable'])['value'].sum().reset_index()

# Reshaping:
df = df.pivot_table(index=['code_region', 'code_dept'], columns='variable', values='value').reset_index()\
       .rename(columns={'Population municipale': 'pop_municipale', 'Population totale': 'pop_totale'})

# Calcul des colonnes pct:
df['pct_pop_municipale'] = df.pop_municipale / df.pop_totale * 100
df['part_region'] = df.pop_municipale / df.groupby('code_region').transform('sum').pop_municipale * 100

# Intégration des données de surface depuis l'autre csv:
surf = pd.read_csv('surface_departements.csv')
df = df.merge(surf.rename(columns={'code_insee': 'code_dept'}))
df['densite'] = df.pop_municipale / df.surf_km2  # calcul colonne densité

# Esthetic:
df = df.sort_values('code_dept')
df = df.rename(columns={'nom': 'nom_dept'})[[
    'nom_dept', 'code_dept', 'code_region', 'pop_municipale', 'pop_totale',
    'pct_pop_municipale', 'part_region', 'densite'
]]
df.to_csv('result-exo-cc.csv', index=False)

In [2]:
df

,nom_dept,code_dept,code_region,pop_municipale,pop_totale,pct_pop_municipale,part_region,densite
80,Ain,01,84,638425,655171,97.444026,8.06409,110.377766
30,Aisne,02,32,536136,549587,97.552526,8.92538,72.343274
81,Allier,03,84,339384,349336,97.151167,4.28684,45.993224
92,Alpes-de-Haute-Provence,04,93,162565,167331,97.151753,3.2371,23.246818
93,Hautes-Alpes,05,93,141107,146148,96.550757,2.80982,24.768650
...,...,...,...,...,...,...,...,...
8,Val-d'Oise,95,11,1221923,1237218,98.763759,10.0843,975.197925
0,Guadeloupe,971,1,394110,400170,98.485644,100,241.193390
9,Martinique,972,2,376480,382294,98.479181,100,345.711662
29,Guyane,973,3,269352,271829,99.088765,100,3.224112
